In [34]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

X_train = pd.read_csv('X_train').values
y_train = pd.read_csv('y_train').values.flatten()
X_test = pd.read_csv('X_test').values

In [39]:
class data_manager():
    def __init__(self):
        self.data = {}

    def read(self, X_train, y_train, X_test):
        self.data['X_train'] = X_train
        self.data['Y_train'] = y_train
        self.data['X_test'] = X_test

    def find_theta(self):
        class_0_id = []
        class_1_id = []
        X_train = self.data['X_train']
        y_train = self.data['Y_train']

        class_0 = X_train[y_train == 0]
        class_1 = X_train[y_train == 1]

        mean_0 = np.mean(class_0, axis=0)
        mean_1 = np.mean(class_1, axis=0)

        n = class_0.shape[1]
        cov_0 = np.zeros((n, n))
        cov_1 = np.zeros((n, n))

        for i in range(class_0.shape[0]):
            cov_0 += np.dot(np.transpose([class_0[i] - mean_0]),
                            [(class_0[i] - mean_0)]) / class_0.shape[0]

        for i in range(class_1.shape[0]):
            cov_1 += np.dot(np.transpose([class_1[i] - mean_1]),
                            [(class_1[i] - mean_1)]) / class_1.shape[0]

        cov = (cov_0*class_0.shape[0] + cov_1*class_1.shape[0]
               ) / (class_0.shape[0] + class_1.shape[0])

        self.w = np.transpose(((mean_0 - mean_1)).dot(inv(cov)))
        self.b = (- 0.5) * (mean_0).dot(inv(cov)).dot(mean_0)\
            + 0.5 * (mean_1).dot(inv(cov)).dot(mean_1)\
            + np.log(float(class_0.shape[0]) / class_1.shape[0])

        result = self.func(X_train)
        answer = self.predict(result)

    def func(self, x):
        arr = np.empty([x.shape[0], 1], dtype=float)
        for i in range(x.shape[0]):
            z = x[i, :].dot(self.w) + self.b
            z *= (-1)
            arr[i][0] = 1 / (1 + np.exp(z))
        return np.clip(arr, 1e-8, 1-(1e-8))

    def predict(self, x):
        ans = np.ones([x.shape[0], 1], dtype=int)
        for i in range(x.shape[0]):
            if x[i] > 0.5:
                ans[i] = 0
        return ans

In [40]:
dm = data_manager()
dm.read(X_train,y_train,X_test)
dm.find_theta()
result = dm.func(X_test)
answer = dm.predict(result)
sampleSubmission = pd.read_csv('sample_submission.csv',index_col='id')
sampleSubmission['label'] = answer
sampleSubmission.to_csv('result.csv')